<a href="https://colab.research.google.com/github/MichaelHalaj/Data-Science-CS301/blob/main/Logistic_Regression_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle
#DISCLAIMER: I tried my best and this is all I can come up with

In [2]:
from google.colab import files
files.upload()
#{'kaggle.json': b'{"username":"michaelhalaj","key":"bdb4c5734628864eae3507d48993d1f2"}'}

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"michaelhalaj","key":"bdb4c5734628864eae3507d48993d1f2"}'}

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle competitions download -c avazu-ctr-prediction

 99% 1.18G/1.19G [00:10<00:00, 163MB/s]
100% 1.19G/1.19G [00:10<00:00, 117MB/s]


In [7]:
! unzip avazu-ctr-prediction.zip 

Archive:  avazu-ctr-prediction.zip
  inflating: sampleSubmission.gz     
  inflating: test.gz                 
  inflating: train.gz                


In [8]:
import numpy as np
import pandas as pd

rows = 10000 #chose this number because I don't have all day to load in millions of rows


df = pd.read_csv('train.gz', usecols = ['click', 'site_category', 'banner_pos'], squeeze = True, nrows = rows)


df['site_category'] = pd.factorize(df.site_category)[0] + 1 #converts site_category to int values

df_test = pd.read_csv('test.gz', nrows = rows)

for i in df.columns: #normalizing the dataset where mean = 0 and std = 1
    df[i] = (df[i] - df[i].mean()) / df[i].std()

rowlen = df.shape[0]

df['ones'] = np.ones((rowlen,), dtype=int) #create a row of ones as starting intercept

xs = np.array(df.loc[:,['ones', 'banner_pos']]) #decided to only use banner position as I didn't know what most other features were
print(xs)
ys = np.array(df.loc[:,'click']) 
print(ys)


[[ 1.         -0.4838297 ]
 [ 1.         -0.4838297 ]
 [ 1.         -0.4838297 ]
 ...
 [ 1.          1.98594926]
 [ 1.         -0.4838297 ]
 [ 1.         -0.4838297 ]]
[-0.45350887 -0.45350887 -0.45350887 ...  2.20480805 -0.45350887
 -0.45350887]


In [9]:
from typing import List

Vector = List[float] #providing a name to a list of floats for readability

epsilon = 1e-5 #this small value is used to prevent 0 from occurring in log functions, which threw off the results since log(0) is undefined

def logistic(x: float) -> float:
  """Also known as a sigmoid function, where as input get large and positive, it gets closer
  and closer to 1 and input gets large and negative, it gets closer to 0"""
  return 1.0 / (1 + np.exp(-x))


def logistic_prime(x :float) -> float:
  """Derivative of the logistic function"""
  y = logistic(x)
  return y * (1 - y)


def _negative_log_likelihood(x: Vector, y: float, beta: Vector) -> float:
  """Working with negative log likelihood, since maximizing the likelihood is the same
  as minimizing its negative; this is the negative log likelihood for one data point"""
  if y == 1:
    return -np.log(epsilon + logistic(np.dot(x, beta))) #epsilon used to prevent log(0) == undefined
  else:
    return -np.log(epsilon + 1 - logistic(np.dot(x, beta))) #epsilon used to prevent log(0) == undefined


def negative_log_likelihood(xs: List[Vector], ys: List[float], beta: Vector) -> float:
  """Sum of all individual log likelihoods within the dataset"""
  return sum(_negative_log_likelihood(x, y, beta) for x,y in zip(xs, ys))


def _negative_log_partial_j(x: Vector, y: float, beta: Vector, j: int) -> float:
  """jth partial derivative for one data point"""
  return -(y - logistic(np.dot(x, beta))) * x[j]


def _negative_log_gradient(x: Vector, y: float, beta: Vector) -> Vector:
  """Gradient for one data point"""
  return [_negative_log_partial_j(x, y, beta, j) for j in range(len(beta))]


def negative_log_gradient(xs: List[Vector], ys: List[float], beta: Vector) -> Vector:
  """Vector sum of gradients for all individual data points"""
  return np.sum([_negative_log_gradient(x, y, beta) for x, y in zip(xs, ys)])


def gradient_step(v: Vector, gradient: Vector, step_size: float) -> Vector:
  """Moves step_size, which in this case is the negative learning rate, in the
  gradient direction from v, which in this case is beta"""
  step = np.multiply(step_size, gradient)
  return np.add(v, step)


import tqdm #used to show progress of the gradient descent within epochs
import random #used to select a random number to start the gradient descent

def predict(row, coefficients):
  y_hat = coefficients[0]
  for col in range(len(row) -1):
    y_hat+=coefficients[col + 1] * row[i]
  return logistic(y_hat)


random.seed(0)
x_train = xs
y_train = ys

learning_rate = 0.0001


beta = [random.random() for _ in range(len(xs[0]))]
with tqdm.trange(1000) as t: #used to show progress of the function
  for epoch in t:
    gradient = negative_log_gradient(x_train, y_train, beta)
    beta = gradient_step(beta, gradient, -learning_rate)
    loss = negative_log_likelihood(x_train, y_train, beta)
    t.set_description(f"loss: {loss:.3f} beta: {beta}")
  


loss: 509.107 beta: [-5.36885496 -5.45532241]: 100%|██████████| 1000/1000 [02:33<00:00,  6.50it/s]


In [21]:
dftest = pd.read_csv('test.gz', nrows = rows)
#print(dftest.columns)
def scale(data : List[Vector]):
  return np.mean(data), np.std(data)
mean, std = scale(x_train)
print(mean, std)

beta_unscaled = [(beta[0]
                  -beta[1] * mean / std),
                  beta[1] / std]
print(beta_unscaled)

df_test['ones'] = np.ones((rowlen,), dtype=int) 
x_test = np.array(df_test.loc[:,['ones','banner_pos']])

def predict(row, coefficients):
  y_hat = coefficients[0]
  for i in range(len(row) -1):
    y_hat+=coefficients[i + 1] * row[i]
  x = logistic(y_hat)

  return x

y_test = []
for row in x_test:
  #print(row)
  y_test.append(1)
  x = (predict(row, beta_unscaled))
  print(x)
  y_test.append(x)

true_positives = false_positives = true_negatives = false_negatives = 0
for x_i, y_i in zip(x_test, y_test):
  prediction = logistic(np.dot(beta, x_i))
  if y_i == 1 and prediction >= 0.5:
    true_positives +=1
  elif y_i == 1:
    false_negatives += 1
  elif prediction >= 0.5:
    false_positives += 1
  else:
    true_negatives += 1
  
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)

Streaming output truncated to the last 5000 lines.
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.00019968059179757816
0.0001

ZeroDivisionError: ignored